In [3]:
from sympy import *

In [4]:
def sympy_to_text(expr):
    # return str(expr).replace('*', ' * ').replace('+', ' + ').replace('/', ' / ')
    return str(expr)

def norm(vector):
    return sqrt(vector.dot(vector))

# sympy code generation tutorial https://www.sympy.org/scipy-2017-codegen-tutorial/notebooks/07-the-hard-way.html
# https://stackoverflow.com/questions/65587417/sympy-generate-c-code-convert-rational-to-float

from sympy.utilities.codegen import codegen, InputArgument
from sympy.codegen.ast import Assignment

class CodeGenerator:
    def __init__(self):
        self.tabs = 0
        self.text = ""

    def tab(self):
        self.tabs += 1

    def untab(self):
        self.tabs -= 1

    def __iadd__(self, text):
        self.text += "\t" * self.tabs + text + "\n"
        return self

    def ccode_print(self, statement):
        for line in ccode(statement).split("\n"):
            self += line


    def print_matrix_to_out(self, matrix, rows, columns):
        out = MatrixSymbol("out", rows, columns)
        sub_exprs, simplified = cse(matrix)
        for var, sub_expr in sub_exprs:
            self += 'f32 ' + ccode(Assignment(var, sub_expr))
            
        out = MatrixSymbol('out', *matrix.shape)
        self.ccode_print(Assignment(out, simplified[0]))

    def return_vec(self, matrix, normalize = False):
        self += "Vec3 m(0.0f);"
        self += "f32* out = m.data();"
        self.print_matrix_to_out(matrix, 3, 1)
        self += "return m" + (".normalized()" if normalize else "") + ";"

    def function_begin(self, return_type, type_name, function_name, arguments):
        self += f"{return_type} {type_name}::{function_name}({arguments}) const {{"
        self.tab()

    def surface_function_begin(self, return_type, type_name, function_name):
        self.function_begin(return_type, type_name, function_name, "f32 u, f32 v")

    def curve_function_begin(self, return_type, type_name, function_name):
        self.function_begin(return_type, type_name, function_name, "f32 t")

    def function_end(self):
        self.untab()
        self += "}"

    def vector_function(self, type_name, function_name, return_value, arguments, normalize = False):
        self.function_begin("Vec3", type_name, function_name, arguments)
        self.return_vec(return_value, normalize)
        self.function_end()

    def mat2_function(self, type_name, function_name, return_value):
        self.function_begin("Mat2", type_name, function_name)
        self += "Mat2 m(Vec2(0.0f), Vec2(0.0f));"
        self += "f32* out = m.data();"
        self.print_matrix_to_out(return_value, 2, 2)
        self += "return m;"
        self.function_end()

    def f32_function(self, type_name, function_name, return_value, arguments):
        self.function_begin("f32", type_name, function_name, arguments)
        self += "f32 out;"
        for line in ccode(return_value, assign_to='out').split("\n"):
            self += line
        self += "return out;"
        self.function_end()

    def newline(self):
        self.text += "\n"


In [5]:
u, v = symbols('u, v')

def surface_data(x, y, z, print_formulas, generate_code, surface_name):
    surface = Matrix([[x], [y], [z]])

    coordinates = [u, v]
    jacobian = simplify(surface.jacobian(Matrix(coordinates)))
    metric = simplify(jacobian.T * jacobian)

    metricInv = simplify(metric.inv())

    coorindate_count = 2
    christoffel = [ 
        [ 
            [Integer(0), Integer(0)],
            [Integer(0), Integer(0)] 
        ],
        [ 
            [Integer(0), Integer(0)],
            [Integer(0), Integer(0)] 
        ] 
    ]

    # https://math.stackexchange.com/questions/1720424/how-to-best-calculate-christoffel-symbols-of-metric-du2-g2u-dv2
    for i in range(coorindate_count):
        for j in range(coorindate_count):
            x_j = coordinates[j]
            for k in range(coorindate_count):
                x_k = coordinates[k]
                for m in range(coorindate_count):
                    x_m = coordinates[m]
                    # This formula
                    # https://youtu.be/8sVDceI70HM?si=P6eGowUsZsxm_wsL&t=120
                    # is equivalent to the one from math.stackechange
                    # The metric tensor is just the jacobian dotted with itself so it contains these dot products.
                    christoffel[i][j][k] += metricInv[i, m] * surface.diff(x_j, x_k).dot(surface.diff(x_m))

    for i in range(coorindate_count):
        christoffel[i] = trigsimp(simplify(Matrix(christoffel[i])))

    x_u = jacobian.col(0)
    x_v = jacobian.col(1)

    x_uu = simplify(x_u.diff(u))
    x_vv = simplify(x_v.diff(v))
    x_uv = simplify(x_u.diff(v))

    # https://stackoverflow.com/questions/39691325/display-a-matrix-with-putting-a-common-factor-in-sympy
    normal = simplify(x_u.cross(x_v))
    normal = expand_trig(normal)
    normal /= gcd(tuple(normal))
    normal = simplify(normal)

    second_fundamental_form = Matrix([
        [ x_u.diff(u).dot(normal), x_u.diff(v).dot(normal) ],
        [ x_v.diff(u).dot(normal), x_v.diff(v).dot(normal) ]
    ]) / norm(normal)
    second_fundamental_form = simplify(second_fundamental_form)
    second_fundamental_form = trigsimp(second_fundamental_form)
    # second_fundamental_form = None

    # # https://en.wikipedia.org/wiki/Differential_geometry_of_surfaces
    shape_operator = metricInv * second_fundamental_form

    eigen = shape_operator.eigenvects()
    if len(eigen) == 1:
        # Repeated eigenvector
        # A symmetric matrix is always diagonalizable so any vector is an eigenvector
        c0 = eigen[0][0][0]
        c1 = c0
        d0 = Matrix([[1], [0]])
        d1 = Matrix([[0], [1]])
    elif len(eigen) == 2:
        c0 = eigen[0][0]
        c1 = eigen[1][0]
        d0 = eigen[0][2][0]
        d1 = eigen[1][2][0]
    else:
        assert False

    gaussian_curvature = simplify(c0 * c1)

    if print_formulas:
        print('christoffel symbols')
        display(christoffel[0])
        display(christoffel[1])

        print('x_u')
        display(x_u)

        print('x_v')
        display(x_v)

        print('x_uu')
        display(x_uu)
        print('x_vv')
        display(x_vv)
        print('x_uv')
        display(x_uv)

        print('normal')
        display(normal)

        print('first fundamental form')
        display(metric)

        print('second fundamental form')
        display(second_fundamental_form)

        print('principal curvatures')
        display(c0)
        display(d0)
        display(c1)
        display(d1)

        print('gaussian curvature')
        display(gaussian_curvature)
    
    if generate_code:
        c = CodeGenerator()
        vector_functions = [
            ('position', surface),
            ('tangentU', x_u),
            ('tangentV', x_v),
            ('xUu', x_uu),
            ('xVv', x_vv),
            ('xUv', x_uv),
        ]
        for name, vector in vector_functions:
            c.vector_function(surface_name, name, vector)
            c.newline()
        c.vector_function(surface_name, "normal", normal, True)
        c.newline()
        
        c.function_begin("ChristoffelSymbols", surface_name, "christoffelSymbols")
        
        out = MatrixSymbol("out", *christoffel[0].shape)
        sub_exprs, simplified = cse([christoffel[0], christoffel[1]])
        for var, sub_expr in sub_exprs:
            c += 'f32 ' + ccode(Assignment(var, sub_expr))

        c += "Mat2 x = Mat2::identity, y = Mat2::identity;"    
        c += "f32* out = x.data();"
        c.ccode_print(Assignment(out, simplified[0]))
        c += "out = y.data();"
        c.ccode_print(Assignment(out, simplified[1]))

        c += "return { .x = x, .y = y };"
        c.function_end()

        mat2_functions = [
            ('firstFundamentalForm', metric),
            ('secondFundamentalForm', second_fundamental_form),
        ]
        for name, mat2 in mat2_functions:
            c.mat2_function(surface_name, name, mat2)
            c.newline()

        c.f32_function(surface_name, 'curvature', gaussian_curvature)
        c += ccode(c0)
        c += ccode(c1)
        print(c.text)

In [6]:
# def curve_data(x, y, z, t):
#     curve = Matrix([x, y, z])
#     tangent = curve.diff(t)



In [7]:
# surface_data(
#     (1 + Rational(1, 2) * v * cos(Rational(1, 2) * u)) * cos(u),
#     (1 + Rational(1, 2) * v * cos(Rational(1, 2) * u)) * sin(u),
#     Rational(1, 2) * v * sin(Rational(1, 2) * u),
#     False, True, "MobiusStrip"
# )

In [8]:
# r, R = symbols('r, R')
# surface_data(
#     (R + r * cos(v)) * cos(u),
#     (R + r * cos(v)) * sin(u),
#     r * sin(v),
#     True, False, "Torus"
# )

In [9]:
# r = symbols('r')
# surface_data(
#     r * (1 + cos(v)) * cos(v),
#     r * (1 + cos(v)) * sin(v),
#     -tanh(u - pi) * r * sin(v),
#     True, False, "ProjectivePlane"
# )
# r = symbols('r')
# real
# surface_data(
#     r * (1 + cos(v)) * cos(v),
#     r * (1 + cos(v)) * sin(v),
#     -tanh(u - pi) * r * sin(v),
#     True, False, "ProjectivePlane"
# )

In [10]:
# r = symbols('r')
# surface_data(
#     r * sech(u) * cos(v),
#     r * sech(u) * sin(v),
#     r * u - r * tanh(u),
#     True, True, "Pseudosphere"
# )

In [11]:
# surface_data(
#     u * v,
#     u,
#     v ** 2,
#     True, True, "WhitneyUmbrella"
# )

In [12]:
# a = Rational(5, 4) * (1 - v / (2 * pi))
# b = a * (1 + cos(u))
# surface_data(
#     (b + 1) * cos(2 * v),
#     (b + 1) * sin(2 * v),
#     10 * v / (2 * pi) + a * sin(u) + 15,
#     True, True, "Seashell"
# )

In [13]:
# surface_data(
#     u,
#     u**2 - v**2,
#     v,
#     True, True, "HyperbolicParaboloid"
# )

In [14]:
# a = Rational(5, 4) * (1 - v / (2 * pi))
# b = a * (1 + cos(u))
# surface_data(
#     (b + 1) * cos(2 * v),
#     (b + 1) * sin(2 * v),
#     10 * v / (2 * pi) + a * sin(u) + 15,
#     True, True, "Seashell"
# )

In [15]:
# surface_data(
#     u,
#     u**3 - 3 * u * v**2,
#     v,
#     True, True, "MonkeySaddle"
# )

In [16]:
# c = symbols('c')
# surface_data(
#     c * cosh(v / c) * cos(u),
#     c * cosh(v / c) * sin(u),
#     v,
#     True, True, "Catenoid"
# )

In [17]:
# c = symbols('c')
# surface_data(
#     u * (1 - Rational(1,3) * u**2 + v**2),
#     v * (1 - Rational(1,3) * v**2 + u**2),
#     (u ** 2 - v ** 2),
#     True, True, "EnneperSurface"
# )

In [18]:
# https://en.wikipedia.org/wiki/Surface_(mathematics)
# https://en.wikipedia.org/wiki/Parametric_surface

# https://math.stackexchange.com/questions/4965132/good-source-or-list-of-3d-parametric-u-v-surfaces

In [19]:
# https://en.wikipedia.org/wiki/Trefoil_knot

In [20]:
def curve_to_surface(curve, curve_time_parameter, surface_radius_parameter):
    display(curve)
    tangent = curve.diff(curve_time_parameter)
    tangent /= norm(tangent)
    tangent = trigsimp(tangent)
    tangent = simplify(tangent)
    display(tangent)

    normal = tangent.diff(curve_time_parameter)
    normal /= norm(normal)
    display(normal)
    normal = simplify(normal)

    display(normal)
    # binormal = tangent.cross(normal)
    # binormal = simplify(binormal)

    # display(tangent)
    # display(normal)
    # display(binormal)

# t, r = symbols('t, r')
# curve_to_surface(Matrix([sin(t) + 2 * sin(2 * t), cos(t) - 2 * cos(2 * t), -sin(3 * t)]), t, r)

In [21]:
# def norm(vector):
#     return sqrt(vector.dot(vector))

# def curve_data(x, y, z, t):
#     curve = Matrix([x, y, z])
#     display(curve)
#     tangent = curve.diff(t)
#     # tangent /= norm(tangent)
#     tangent /= gcd(tuple(tangent))
#     tangent = trigsimp(tangent)
#     tangent = simplify(tangent)
#     display(tangent)

#     # normal = tangent.diff(t)
#     # normal /= norm(normal)
#     # display(normal)
#     # normal = simplify(normal)

#     # display(normal)
#     # binormal = tangent.cross(normal)
#     # binormal = simplify(binormal)

#     # display(tangent)
#     # display(normal)
#     # display(binormal)

# t, r = symbols('t, r')
# curve_data(sin(t) + 2 * sin(2 * t), cos(t) - 2 * cos(2 * t), -sin(3 * t), t)

In [27]:
t = symbols('t', real=True)

def curve_info(x, y, z, curve_name):
    # https://en.wikipedia.org/wiki/Frenet%E2%80%93Serret_formulas#Other_expressions_of_the_frame
    position = Matrix([x, y, z])
    unnormalized_tangent = position.diff(t)
    unnormalized_tangent = simplify(unnormalized_tangent)

    first_derivative = unnormalized_tangent

    second_derivative = first_derivative.diff(t)
    second_derivative = simplify(second_derivative)

    third_derivative = second_derivative.diff(t)
    third_derivative = simplify(third_derivative)

    print('unnormalized tangent')
    display(unnormalized_tangent)

    tangent = unnormalized_tangent / norm(unnormalized_tangent)
    tangent = trigsimp(tangent)
    tangent = simplify(tangent)

    print('tangent')
    display(tangent)

    unnormalized_normal = tangent.diff(t)
    unnormalized_normal = simplify(unnormalized_normal)

    print('unnormalized normal')
    display(unnormalized_normal)

    first_cross_second = simplify(first_derivative.cross(second_derivative))
    first_cross_second_norm = norm(first_cross_second)
    first_cross_second_norm = simplify(first_cross_second_norm)
    curvature = first_cross_second_norm / (norm(first_derivative) ** 3)
    # curvature = expand_trig(curvature)
    curvature = simplify(curvature)
    curvature = trigsimp(curvature)

    print('curvature')
    display(curvature)

    normal = unnormalized_normal / simplify(norm(unnormalized_normal))
    normal = simplify(normal)

    print('normal')
    display(normal)

    binormal = tangent.cross(normal)
    binormal = simplify(binormal)

    print('binormal')
    display(binormal)

    torsion = first_cross_second.dot(third_derivative) / (first_cross_second_norm ** 2)
    # torsion = expand_trig(torsion)
    torsion = trigsimp(torsion)
    torsion = simplify(torsion)

    print('torsion')
    display(torsion)

    # print(simplify(norm(tangent)), simplify(norm(normal)), simplify(norm(binormal)))

    curve_arguments = 'f32 t'
    c = CodeGenerator()
    
    vector_functions = [
        ('position', position),
        ('tangent', tangent),
        ('unnormalizedTangent', unnormalized_tangent),
        ('normal', normal),
        ('unnormalizedNormal', unnormalized_normal),
        ('binormal', binormal),
    ]
    for name, vector in vector_functions:
        c.vector_function(curve_name, name, vector, curve_arguments)
        c.newline()
    
    f32_functions = [
        ('curvature', curvature),
        ('torsion', torsion),
    ]
    for name, f32 in f32_functions:
        c.f32_function(curve_name, name, f32, curve_arguments)
        c.newline()

    print(c.text)

    

In [23]:
# a = symbols('a', real=True, positive=True)
# b = symbols('b', real=True)
# curve_info(a * cos(t), a * sin(t), b * t, 'Helix')

In [24]:
# r = symbols('r', real=True, positive=True)
# curve_info(r * (1 + cos(t)), r * sin(t), 2 * r * sin(Rational(1, 2) * t), 'VivanisCurve')

In [29]:
# r = symbols('r', real=True, positive=True)
# curve_info(r * (t - sin(t)), r * (1 - cos(t)), 0, 'Cycloid')
# Sympy cannot simplify the absolute values correctly

In [ ]:
# r = symbols('r', real=True, positive=True)
# curve_info(r * (t - sin(t)), r * (1 - cos(t)), 0, 'Cycloid')
# Sympy cannot simplify the absolute values correctly

In [ ]:
a, b = symbols('a, b', real=True)
curve_info(a * cos(t) + b * cos(3 * t), a * sin(t) - b * sin(3 * t), 2 * sqrt(a * b) * sin(2 * t), 'TenisBallCurve')

unnormalized tangent


Matrix([
[-a*sin(t) - 3.0*b*sin(3.0*t)],
[ a*cos(t) - 3.0*b*cos(3.0*t)],
[                           0]])

tangent


Matrix([
[-(0.333333333333333*a*sin(t) + 1.0*b*sin(3.0*t))/sqrt(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2)],
[ (0.333333333333333*a*cos(t) - 1.0*b*cos(3.0*t))/sqrt(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2)],
[                                                                                                                          0]])

unnormalized normal


Matrix([
[  (1.33333333333333*a*b*(0.333333333333333*a*sin(t) + 1.0*b*sin(3.0*t))*sin(4.0*t) - (0.333333333333333*a*cos(t) + 3.0*b*cos(3.0*t))*(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2))/(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2)**(3/2)],
[(-1.33333333333333*a*b*(0.333333333333333*a*cos(t) - 1.0*b*cos(3.0*t))*sin(4.0*t) + (-0.333333333333333*a*sin(t) + 3.0*b*sin(3.0*t))*(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2))/(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2)**(3/2)],
[                                                                                                                                                                                                                                                                                         0]])

curvature


1.0*Abs(0.037037037037037*a**2 + 0.222222222222222*a*b*cos(4.0*t) - b**2)/(0.111111111111111*a**2 - 0.666666666666667*a*b*cos(4.0*t) + 1.0*b**2)**(3/2)

KeyboardInterrupt: 